# Perform variant quality control

In this notebook, we perform variant quality control using [PLINK2](https://www.cog-genomics.org/plink/2.0/).

Note that this work is part of a larger project to [Demonstrate the Potential for Pooled Analysis of All of Us and UK Biobank Genomic Data](https://docs.google.com/document/d/19ZS0z_-7FEM37pNDAXaWaqBSLnqyd9MZEkiOmtF3n_0/edit#). Specifically this is for the portion of the project that is the **pooled** analysis.

# Setup 

<div class="alert alert-block alert-warning">
    <b>Cloud Environment</b>: This notebook was written for use on the <i>All of Us</i> Workbench.
    <ul>
        <li>Use "Recommended Environment" <kbd><b>General Analysis</b></kbd> which creates compute type <kbd><b>Standard VM</b></kbd> with reasonable defaults for CPU, RAM and disk.</li>
        <li>This notebook can take a while to run <b>TBD DETAILS CHR21 VS. ALL CHRS</b>. Recommend that it is run in the background via <kbd>run_notebook_in_the_background</kbd>.</li>    </ul>
</div>

In [ ]:
from datetime import datetime
import os
import time

## Setup plink2

https://www.cog-genomics.org/plink/2.0/

In [ ]:
%%bash

##### plink 2 install
PLINK_VERSION=2.3.Alpha
PLINK_ZIP_PATH=/tmp/plink-$PLINK_VERSION.zip
curl -L -o $PLINK_ZIP_PATH https://s3.amazonaws.com/plink2-assets/alpha2/plink2_linux_x86_64.zip
mkdir -p /tmp/plink2/
unzip -o $PLINK_ZIP_PATH -d /tmp/plink2/

In [ ]:
!/tmp/plink2/plink2 --version # --help

## Define constants

In [ ]:
# Papermill parameters. See https://papermill.readthedocs.io/en/latest/usage-parameterize.html

#---[ Inputs ]---
# The BGEN file created via aou_workbench_pooled_analyses/05_write_pooled_bgen.ipynb.
REMOTE_MERGED_BGEN = 'gs://fc-secure-e53e4a44-7fe2-42b7-89b7-01aae1e399f7/data/pooled/geno/20220215/aou-alpha3-ukb-chr1-chr22.bgen'
# The sample file created via aou_workbench_pooled_analyses/05_write_pooled_bgen.ipynb.
REMOTE_MERGED_BGEN_SAMPLE = 'gs://fc-secure-e53e4a44-7fe2-42b7-89b7-01aae1e399f7/data/pooled/geno/20220215/aou-alpha3-ukb-chr1-chr22.sample'
# This TSV was created via notebook aou_workbench_pooled_analyses/02_merge_lipids_phenotypes.ipynb
REMOTE_PHENOTYPE_SAMPLE_IDS = 'gs://fc-secure-fd6786bf-6c28-4f33-ac30-3860fbeee5bb/data/pooled/phenotypes/20211224/aou_alpha2_ukb_pooled_lipids_phenotype.tsv'

#---[ Outputs ]---
# Create a timestamp for a folder of results generated today.
DATESTAMP = time.strftime('%Y%m%d')
OUTPUT_FILENAME_PREFIX = 'aou_alpha3_ukb_lipids'
OUTPUT_FOLDER = f'{os.getenv("WORKSPACE_BUCKET")}/data/pooled/variant-qc/{DATESTAMP}/'

In [ ]:
LOCAL_MERGED_BGEN = os.path.basename(REMOTE_MERGED_BGEN)
LOCAL_MERGED_BGEN_SAMPLE = os.path.basename(REMOTE_MERGED_BGEN_SAMPLE)
LOCAL_PHENOTYPE_SAMPLE_IDS = os.path.basename(REMOTE_PHENOTYPE_SAMPLE_IDS)

## Copy data locally

In [ ]:
!gsutil cp -n {REMOTE_MERGED_BGEN} {REMOTE_MERGED_BGEN_SAMPLE} .

In [ ]:
!gsutil cp {REMOTE_PHENOTYPE_SAMPLE_IDS} .

# Variant QC for regenie step 1 via plink2

Use plink2 to perform [variant QC appropriate for step 1 of regenie](https://rgcgithub.github.io/regenie/recommendations/#exclusion-files), obtaining a subset of high quality variants roughly equal to the number of samples.

In [ ]:
!/tmp/plink2/plink2 \
  --bgen {LOCAL_MERGED_BGEN} ref-first \
  --sample {LOCAL_MERGED_BGEN_SAMPLE} \
  --chr 1-22 \
  --keep {LOCAL_PHENOTYPE_SAMPLE_IDS} \
  --geno 0.1 \
  --mind 0.1 \
  --mac 100 \
  --hwe 1e-15 \
  --write-snplist \
  --write-samples \
  --no-id-header \
  --out {OUTPUT_FILENAME_PREFIX}_step1QC_plink

In [ ]:
!ls -lth . | head

In [ ]:
!head {OUTPUT_FILENAME_PREFIX}_step1QC_plink.id

In [ ]:
!wc -l {OUTPUT_FILENAME_PREFIX}_step1QC_plink.id

In [ ]:
!head {OUTPUT_FILENAME_PREFIX}_step1QC_plink.snplist

In [ ]:
!wc -l {OUTPUT_FILENAME_PREFIX}_step1QC_plink.snplist

In [ ]:
!ls {OUTPUT_FILENAME_PREFIX}_step1QC_plink*

In [ ]:
!gsutil -m cp {OUTPUT_FILENAME_PREFIX}_step1QC_plink* {OUTPUT_FOLDER}

# Variant QC for regenie step 2 via plink2

Use plink2 to perform [variant QC appropriate for step 2 of regenie](https://rgcgithub.github.io/regenie/recommendations/#exclusion-files), obtaining a subset of high quality variants.

In [ ]:
!/tmp/plink2/plink2 \
  --bgen {LOCAL_MERGED_BGEN} ref-first \
  --sample {LOCAL_MERGED_BGEN_SAMPLE} \
  --chr 1-22 \
  --keep {LOCAL_PHENOTYPE_SAMPLE_IDS} \
  --geno 0.1 \
  --mind 0.1 \
  --hwe 1e-15 \
  --write-snplist \
  --write-samples \
  --no-id-header \
  --out {OUTPUT_FILENAME_PREFIX}_step2QC_plink

# This differs from step 1 QC in that the following parameter was removed
#    --mac 100 \

In [ ]:
!ls -lth {OUTPUT_FILENAME_PREFIX}*

In [ ]:
!wc -l {OUTPUT_FILENAME_PREFIX}_step2QC_plink.id

In [ ]:
!wc -l {OUTPUT_FILENAME_PREFIX}_step2QC_plink.snplist

In [ ]:
!ls {OUTPUT_FILENAME_PREFIX}_step2QC_plink*

In [ ]:
!gsutil -m cp {OUTPUT_FILENAME_PREFIX}_step2QC_plink* {OUTPUT_FOLDER}

In [ ]:
!gsutil ls {OUTPUT_FOLDER}

# Provenance 

In [ ]:
%%bash

date

In [ ]:
%%bash

pip3 freeze